In [1]:
import pandas as pd
import numpy as np

C:\Users\rohit\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
extracted_data = pd.read_csv('data.csv')

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

def scaling(dataframe):
    scaler = StandardScaler()
    prep_data = scaler.fit_transform(dataframe.iloc[:, 6:15].to_numpy())
    return prep_data, scaler

def nn_predictor(prep_data):
    neigh = NearestNeighbors(metric='cosine', algorithm='brute')
    neigh.fit(prep_data)
    return neigh

def build_pipeline(neigh, scaler, params):
    transformer = FunctionTransformer(neigh.kneighbors, kw_args=params)
    pipeline = Pipeline([('std_scaler', scaler), ('NN', transformer)])
    return pipeline

def extract_data(dataframe, ingredient_filter):
    extracted_data = dataframe.copy()
    if ingredient_filter is not None:
        for ingredient in ingredient_filter:
            extracted_data = extracted_data[extracted_data['RecipeIngredientParts'].str.contains(ingredient, regex=False)]
    return extracted_data

def apply_pipeline(pipeline, _input, extracted_data):
    print('apply pipeline', extracted_data.iloc[pipeline.transform(_input)[0]])
    return extracted_data.iloc[pipeline.transform(_input)[0]]

def recommend(dataframe, _input, ingredient_filter=None, params={'return_distance': False}):
    extracted_data = extract_data(dataframe, ingredient_filter)
    prep_data, scaler = scaling(extracted_data)
    neigh = nn_predictor(prep_data)
    pipeline = build_pipeline(neigh, scaler, params)
    return apply_pipeline(pipeline, _input, extracted_data)


In [12]:
extracted_data.iloc[0:1,6:15].to_numpy()

array([[ 536.1,   24. ,    3.8,    0. , 1558.6,   64.2,   17.3,   32.1,
          29.3]])

In [13]:
extracted_data.iloc[0:1,:15]

,RecipeId,Name,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,41,Carina's Tofu-Vegetable Kebabs,PT20M,PT24H,PT24H20M,"c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3


In [16]:
test_input = extracted_data.iloc[0:1, 6:15].to_numpy()
recommend(extracted_data, test_input, ingredient_filter=None, params={'return_distance': False})

apply pipeline        RecipeId                                               Name CookTime  \
0            41                     Carina's Tofu-Vegetable Kebabs    PT20M   
21398    406551  Sweet &amp; Smoky Paprika  Chicken Chili-Slow ...     PT4H   
18792    376451  Roasted Garlic, Sun-Dried Tomato, and White Be...    PT55M   
21437    406895                      Pf Chang's Tofu Lettuce Wraps    PT10M   
20611    396863                                Awesome Baked Beans    PT55M   

      PrepTime TotalTime                              RecipeIngredientParts  \
0        PT24H  PT24H20M  c("extra firm tofu", "eggplant", "zucchini", "...   
21398    PT30M   PT4H30M  c("boiling water", "olive oil", "yellow onion"...   
18792    PT10M    PT1H5M  c("head of garlic", "water", "sun-dried tomato...   
21437    PT50M      PT1H  c("extra firm tofu", "brown sugar", "soy sauce...   
20611    PT10M    PT1H5M  c("ground beef", "catsup", "dark brown sugar",...   

       Calories  FatContent  Satura

,RecipeId,Name,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions
0,41,Carina's Tofu-Vegetable Kebabs,PT20M,PT24H,PT24H20M,"c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,"c(""Drain the tofu, carefully squeezing out exc..."
21398,406551,Sweet &amp; Smoky Paprika Chicken Chili-Slow ...,PT4H,PT30M,PT4H30M,"c(""boiling water"", ""olive oil"", ""yellow onion""...",449.0,14.4,3.9,37.5,1013.9,57.4,14.1,18.7,26.2,"c(""Allow red chiles and prunes to steep in 2 c..."
18792,376451,"Roasted Garlic, Sun-Dried Tomato, and White Be...",PT55M,PT10M,PT1H5M,"c(""head of garlic"", ""water"", ""sun-dried tomato...",477.8,15.9,2.3,0.0,1274.4,69.9,18.8,18.9,22.6,"c(""Preheat oven to 375 degrees."", ""Remove whit..."
21437,406895,Pf Chang's Tofu Lettuce Wraps,PT10M,PT50M,PT1H,"c(""extra firm tofu"", ""brown sugar"", ""soy sauce...",726.1,27.1,3.4,0.0,4139.6,106.0,27.4,66.8,33.3,"c(""Drain the tofu, and then remove excess mois..."
20611,396863,Awesome Baked Beans,PT55M,PT10M,PT1H5M,"c(""ground beef"", ""catsup"", ""dark brown sugar"",...",411.0,11.8,4.4,50.2,1195.4,60.5,9.7,25.2,20.4,"c(""Preheat oven to 350."", ""In very large mixin..."
